# My Zeebe Dashboard

In [ ]:
from IPython.display import display
from ipywidgets import widgets

## Process list

In [ ]:
create_display= widgets.Output()
display(create_display)

In [ ]:
instances_display = widgets.Output()
display(instances_display)

In [ ]:
instance_display = widgets.Output()
display(instance_display)

## Task list

In [ ]:
tasks_display= widgets.Output()
display(tasks_display)

In [ ]:
task_display = widgets.Output()
display(task_display)

## Decision list

In [ ]:
decisions_display = widgets.Output()
display(decisions_display)

In [ ]:
decision_display = widgets.Output()
display(decision_display)

In [ ]:
from IPython.display import clear_output, JSON
from jupyterlab_form_js import FormJSWidget as Form
from ipydatagrid import DataGrid
from requests import post
from datetime import datetime
import base64
import ipyvuetify as v
import json
import pandas as pd
import requests
import time

In [ ]:
config = { "style": { "height": "400px" }, "zoom": 1.1 }
BPMN = lambda bpmn, update={}: {
    "application/bpmn+xml": bpmn,
    "application/bpmn+json": json.dumps(config | update)
}
DMN = lambda dmn, update={}: {
    "application/dmn+xml": dmn,
    "application/dmn+json": json.dumps(config | update)
}
var = {}

In [ ]:
def query(query, operation_name=None, variables=None):
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "query": query, 
        "operation_name": operation_name,
        "variables": variables or {},
    }
    response = requests.post(
        "http://localhost:8001/graphql",
        headers=headers,
        json=data
    )
    return response.json()["data"]

In [ ]:
QUERY_DEFINITIONS = """\
{
  process(sort: bpmnProcessName) {
    nodes {
      bpmnProcessId
      bpmnProcessName
    }
  }
}
"""

In [ ]:
var["select"] = v.Select(
    v_model=None,
    label='Create new',
    items=[],
)
def render_create():
    select = var["select"]
    select.items = list(set([
        d["bpmnProcessId"] for d
        in query(QUERY_DEFINITIONS)["process"]["nodes"]
    ]))
    btn = v.Btn(color='primary', children=['Create instance'])
    def create_instance(*args):
        results = !zbctl --insecure create instance {var["select"].v_model}
        try:
            key = json.loads("".join(results))["processInstanceKey"]
        except:
            with create_display:
                clear_output()
                display(select)
                display(results)
        for i in range(100):
            time.sleep(0.1)  # wait for zeebe data export
            data = query(QUERY_INSTANCE, "process_instance", {"id": key})
            if data["process_instance_row"]:
                break
        render_instances()
    btn.on_event('click', create_instance)
    with create_display:
        clear_output()
        display(select)
        display(btn)
render_create()

In [ ]:
QUERY_INSTANCES = """\
{
  process_instance(first: 100) {
    nodes {
      key
      processDefinition {
        bpmnProcessName
      }
      created
      tasks: element_instance_list(
        filter: {bpmnElementType: {eq: "USER_TASK"}, completed: {isnull: true}}
      ) {
        totalCount
      }
      incidents: incident_list(filter: {completed: {isnull: true}}) {
        totalCount
      }
    }
  }
}
"""

In [ ]:
QUERY_ACTIVE_INSTANCES = """\
{
  process_instance(filter: {completed: {isnull: true}}, first: 100) {
    nodes {
      key
      processDefinition {
        bpmnProcessName
      }
      created
      tasks: element_instance_list(
        filter: {bpmnElementType: {eq: "USER_TASK"}, completed: {isnull: true}}
      ) {
        totalCount
      }
      incidents: incident_list(filter: {completed: {isnull: true}}) {
        totalCount
      }
    }
  }
}
"""

In [ ]:
QUERY_COMPLETED_INSTANCES = """\
{
  process_instance(filter: {completed: {notnull: true}}, first: 100) {
    nodes {
      key
      processDefinition {
        bpmnProcessName
      }
      created
      tasks: element_instance_list(
        filter: {bpmnElementType: {eq: "USER_TASK"}, completed: {isnull: true}}
      ) {
        totalCount
      }
      incidents: incident_list(filter: {completed: {isnull: true}}) {
        totalCount
      }
    }
  }
}
"""

In [ ]:
QUERY_INSTANCE = """\
query process_instance_($id: String!) {
  process_instance_row(key: $id) {
    processDefinition {
      resource
    }
    incidents: incident_list {
      nodes {
        activityId: elementId
        startTime: created
        endTime: completed
      }
    }
    activities: element_instance_list {
      nodes {
        activityId: elementId
        startTime: created
        endTime: completed
      }
    }
  }
}
"""

In [ ]:
toggle_mode = v.BtnToggle(v_model=0, class_='mr-3', children=[
    v.Btn(text=True, children=["All"]),
    v.Btn(text=True, children=["Running"]),
    v.Btn(text=True, children=["Completed"]),
])
def render_instances():
    def on_change(*args):
        render_instances()
    toggle_mode.on_event('change', on_change)
    data = [
        {
            "key": item["key"],
            "name": item["processDefinition"]["bpmnProcessName"],
            "created": datetime.fromtimestamp(int(item["created"]) / 1000.),
            "tasks": item["tasks"]["totalCount"],
            "incidents": item["incidents"]["totalCount"],
        }
        for item in query(toggle_mode.v_model == 0 and QUERY_INSTANCES or toggle_mode.v_model == 1 and QUERY_ACTIVE_INSTANCES or QUERY_COMPLETED_INSTANCES)["process_instance"]["nodes"]
    ]
    df = pd.DataFrame(data)
    dg = DataGrid(
        df[["name", "created", "tasks", "incidents"]],
        selection_mode="cell",
        column_widths={
            "name": 300,
            "created": 150,
            "tasks": 100,
            "incidents": 100,
        },
        layout={"height": f"{len(df) * 24 + 24}px"}
    )
    def on_select(selection):
        key = df.loc[selection["row"]]["key"]
        data = query(QUERY_INSTANCE, "process_instance", {"id": key})["process_instance_row"]
        bpmn = base64.b64decode(data["processDefinition"]["resource"]).decode("utf-8")
        with instance_display:
            clear_output()
            display(BPMN(bpmn, dict(activities=data["activities"]["nodes"], incidents=data["incidents"]["nodes"])), raw=True)
    dg.on_cell_click(on_select)
    refresh = v.Container(children=[v.Btn(color='primary', children=['Update'])])
    refresh.children[0].on_event('click', lambda *args: render_instances())
    with instances_display:
        clear_output()
        display(toggle_mode)
        display(widgets.HBox([dg, refresh]))
render_instances()

In [ ]:
QUERY_JOBS = """\
{
  job(first: 1000) {
    nodes {
      key
      processInstance {
        processDefinition { bpmnProcessName}
      }
      type
      elementInstance { elementName}
      state
    }
  }
}
"""

In [ ]:
QUERY_JOB = """\
query job($id: String!) {
  job_row(key: $id) {
    key
    type
    elementInstance {
      elementName
    }
    variables
    form {
      schema
    }
  }
}
"""

In [ ]:
def render_tasks():
    data = [
        {
            "key": item["key"],
            "process": item["processInstance"]["processDefinition"]["bpmnProcessName"],
            "type": item["type"],
            "name": item["elementInstance"]["elementName"],
            "state": item["state"]
        }
        for item in query(QUERY_JOBS)["job"]["nodes"]
    ]
    df = pd.DataFrame(data)
    dg = DataGrid(
        df[["name", "type", "state", "process"]],
        selection_mode="cell",
        column_widths={
            "name": 100,
            "process": 150,
            "type": 200,
            "state": 100,
        },
        layout={"height": f"{len(df) * 24 + 24}px"}
    )
    def on_select(selection):
        key = df.loc[selection["row"]]["key"]
        data = query(QUERY_JOB, "job", {"id": key})["job_row"]
        form = Form(
            schema=data["form"]["schema"],
            data=data["variables"],
        )
        def handle_submit(f, data, errors):
            if errors:
                return
            var["key"] = key
            var["variables"] = json.dumps(json.dumps(data))
            results = !zbctl --insecure complete job {var["key"]} --variables {var["variables"]}
            time.sleep(1)
            with task_display:
                clear_output()
        form.on_submit(handle_submit)
        with task_display:
            clear_output()
            display(form)
    dg.on_cell_click(on_select)
    refresh = v.Container(children=[v.Btn(color='primary', children=['Update'])])
    refresh.children[0].on_event('click', lambda *args: render_tasks())
    with tasks_display:
        clear_output()
        display(widgets.HBox([dg, refresh]))
render_tasks()

In [ ]:
QUERY_DECISIONS = """\
{
  decision_evaluation(first: 1000) {
    nodes {
      key
      decision {
        decisionName
      }
      created
      decisionOutput
      processDefinition {
        bpmnProcessName
      }
    }
  }
}
"""

In [ ]:
QUERY_DECISION = """\
query decision_evaluation($id: String!) {
  decision_evaluation_row(key: $id) {
    key
    decision {
      decisionId
      decisionRequirements {
        resource
      }
    }
    evaluatedDecisions
  }
}
"""

In [ ]:
def render_decisions():
    data = [
        {
            "key": item["key"],
            "name": item["decision"]["decisionName"],
            "created": datetime.fromtimestamp(int(item["created"]) / 1000.),
            "process": item["processDefinition"]["bpmnProcessName"],
            "output": str(item["decisionOutput"]),
        }
        for item in query(QUERY_DECISIONS)["decision_evaluation"]["nodes"]
    ]
    df = pd.DataFrame(data)
    dg = DataGrid(
        df[["name", "created", "process", "output"]],
        selection_mode="cell",
        column_widths={
            "name": 100,
            "created": 100,
            "process": 150,
        },
        layout={"height": f"{len(df) * 24 + 24}px"}
    )
    def on_select(selection):
        key = df.loc[selection["row"]]["key"]
        data = query(QUERY_DECISION, "decision_evaluation", {"id": key})["decision_evaluation_row"]
        dmn = base64.b64decode(data["decision"]["decisionRequirements"]["resource"]).decode("utf-8")
        with decision_display:
            clear_output()
            display(DMN(dmn, data["evaluatedDecisions"][0]), raw=True)
    dg.on_cell_click(on_select)
    refresh = v.Container(children=[v.Btn(color='primary', children=['Update'])])
    refresh.children[0].on_event('click', lambda *args: render_decisions())
    with decisions_display:
        clear_output()
        display(widgets.HBox([dg, refresh]))
render_decisions()